In [1]:
from bank import Bank
import pandas as pd
import os

In [2]:
!python --version

Python 3.6.12


In [3]:
os.environ.update({i[0]: i[1] for i in [i.split('=') for i in open('.env').read().split()]})

In [4]:
trs = Bank.flat_transactions()

In [6]:
df = pd.DataFrame(trs)

In [12]:
df = pd.read_csv('~/Documents/transactions/transactions_20210606_170532.csv')